# Базовые операции Apache Impala

## Операции чтения

### Определение версии Apache Impala

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
select version()
"

### Получение списка баз данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW DATABASES
"

Небходимо дождаться, когда базы данных `tpcds_parquet` и `tpcds_raw` станут доступны:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW DATABASES
" |& grep 'tpcds_raw\|tpcds_parquet' ||
echo "Базы данных tpcds_raw и tpcds_parquet еще не загружены, пожалуйста подождите"

### Получение списка баз данных по шаблону

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW DATABASES LIKE '*tpcds*'
"

### Определение текущей базы данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
select current_database()
"

### Получение списка таблиц

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW TABLES
"

### Получение списка таблиц в определенной базе данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW TABLES IN tpcds_parquet
"

### Получение списка таблиц по шаблону

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW TABLES IN tpcds_parquet LIKE 'web*'
"

### Получение информации о таблице

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
DESCRIBE tpcds_parquet.web_page
"

### Извлечение данных из таблицы

#### Подсчёт числа строк

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SELECT count(*)
  FROM tpcds_parquet.customer;
"

#### Подсчёт только уникальных значений

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT count(distinct c_birth_month)
  FROM customer
"

Существует специальная функция [`NDV`](https://impala.apache.org/docs/build/plain-html/topics/impala_ndv.html#ndv) для подсчета уникальных значений:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT NDV(c_birth_month)
  FROM customer
"

#### Ограничение числа элементов результирующего множества

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT distinct c_salutation
  FROM customer;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT DISTINCT c_salutation
  FROM customer
 LIMIT 3;
"

### Соединение таблиц

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
-- Соединение таблиц customer и customer_address, чтобы узнать текущее место жительства клиента
SELECT c.c_first_name, c.c_last_name
     , ca_street_name
     , ca_suite_number
     , ca_city
     , ca_state
     , ca_country
  FROM customer c
  JOIN customer_address a
    ON (a.ca_address_sk = c.c_current_addr_sk)
 LIMIT 5
"

## Операции трансформации

### Создание таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE TABLE my_table (
  id INT,
  name STRING
)
"

### Создание базы данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE my_impala_db
"

### Перемещение таблицы из одной базы данных в другую

#### Получение списка таблиц в новой базе данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW TABLES in my_impala_db
"

#### Перемещение таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
ALTER TABLE my_table
RENAME TO my_impala_db.my_table
"

#### Получение списка таблиц в новой базе данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
SHOW TABLES in my_impala_db
"

### Добавление данных в таблицу

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_impala_db -q "
INSERT INTO my_table VALUES
(1, 'Hello'),
(2, 'Hallo'),
(3, 'Hola'),
(4, 'Bonjour')
"

#### Проверка доступности данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_impala_db -q "
SELECT * FROM my_table
"

### Обновление данных в таблице

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_impala_db -q "
UPDATE my_table
   SET name = 'Привет'
 WHERE id = 1
" || true

Попытка обновления данных заканчивается ошибкой, т.к. обновление данных не поддерживается стандартными средствами Impala.

Данные по ключу `id = 1` остались прежними:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_impala_db -q "
SELECT *
  FROM my_table
 WHERE id = 1
"

### Сортировка результирующего множества

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d my_impala_db -q "
SELECT *
  FROM my_table
 ORDER BY id DESC
"

## Агрегация

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT min(c_birth_year) oldest
     , max(c_birth_year) youngest
     , avg(c_birth_year) average_birth_year
     , sum(year(current_date()) - c_birth_year) total_years
  FROM customer
"

#### Условие на группы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d tpcds_parquet -q "
SELECT count(*)
     , c_birth_day
     , c_birth_month
     , c_birth_year
  FROM customer
 WHERE c_birth_day IS NOT NULL
   AND c_birth_month IS NOT NULL
   AND c_birth_year IS NOT NULL
 GROUP BY c_birth_day, c_birth_month, c_birth_year
HAVING count(*) = 1
 LIMIT 5
"

## Вывод

1. Язык запросов близок к языку запросов ANSI SQL 92, поэтому опытные пользователи традиционных баз данных свободно могут использовать Apache Impala;
1. Модификация данных возможна только через `INSERT`;
1. Операции `UPDATE` и `DELETE` недоступны по умолчанию, для их использования необходимо использовать Kudu таблицы.

## Задания

1. Получить версию текущей базы данных;
1. Сколько баз данных в вашей Impala?
1. Создать базу данных `my_birthday_songs_playlist`;
1. Создать таблицы `songs` и `artists`;
1. Найти самые популярные песни в день вашего рождения с 0 до 14 лет на сайте [birthdayjams](https://www.birthdayjams.com/) и наполнить ими таблицы `songs` и `artists` данными;
1. Вывести список песен вместе с исполнителями в порядке убывания года (более новые песни идут в начале списка);
1. Какой товар является самым старым заказом в `tpcds_parquet.store_sales` (время продажи в колонке `ss_sold_time_sk`)?

### Ответы

1. Получить версию текущей базы данных

<details>
    <summary>Ответ</summary>

```sql
select version()
```
</details>

2. Сколько баз данных в вашей Impala?

<details>
    <summary>Ответ</summary>

Необходимо посчитать число записей, которые выводит следующая команда:

```bash
impala-shell -i "${IMPALA_HOST}" -q "
SHOW DATABASES
"
```
</details>

3. Создать базу данных `my_birthday_songs_playlist`:

<details>
    <summary>Ответ</summary>

```bash
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE my_birthday_songs_playlist
"
```
</details>

4. Создать таблицы `songs` и `artists`:

<details>
    <summary>Ответ</summary>

```bash
impala-shell -i "${IMPALA_HOST}" -d my_birthday_songs_playlist -q "
CREATE TABLE songs (
  id INT,
  year INT,
  name STRING,
  artist_id INT
);

CREATE TABLE artists (
  id INT,
  name STRING
);
"
```
</details>

5. Найти самые популярные песни в день вашего рождения с 0 до 14 лет на сайте birthdayjams и наполнить ими таблицы songs и artists данными:

<details>
    <summary>Ответ</summary>

```bash
impala-shell -i "${IMPALA_HOST}" -d my_birthday_songs_playlist -q "
INSERT INTO songs VALUES
(1, 1989, 'Rock On', 1),
(2, 1990, 'Vogue', 2);

INSERT INTO artists VALUES
(1, 'Michael Damian'),
(2, 'Madonna');
"
```
</details>

6. Вывести список песен вместе с исполнителями в порядке убывания года (более новые песни идут в начале списка):

<details>
    <summary>Ответ</summary>

```sql
SELECT *
  FROM songs
  JOIN artists
    ON (songs.artist_id = artists.id)
 ORDER BY year DESC
```
</details>

7. Какой товар является самым старым заказом в `tpcds_parquet.store_sales` (время продажи в колонке `ss_sold_time_sk`)?

<details>
    <summary>Ответ</summary>

```sql
WITH oldest_sale AS (
    SELECT min(ss_sold_time_sk) ss_sold_time_sk
      FROM store_sales
)
SELECT i.i_product_name
  FROM item i
  JOIN store_sales s ON (s.ss_item_sk = i.i_item_sk)
 WHERE EXISTS (
         SELECT null
           FROM oldest_sale o
          WHERE s.ss_sold_time_sk = o.ss_sold_time_sk
       )
```
</details>